<a href="https://colab.research.google.com/github/HeqingZ/FCOS-MViT_Debug/blob/main/FCOS%2BMViT_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive/")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
#install detectron2
#the detectron2 package requires pyyaml version > 5.3 
!python -m pip install pyyaml==5.3.1
!pip install click
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 269 kB 5.0 MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44636 sha256=478c3a2d59f989d630f65133ec61cbbbd3d4e1547d02388e59b7b53ae430dacb
  Stored in directory: /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653
Successfully built pyyaml
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 6.0
    Uninstalling PyYAML-6.0:
      Successfully uninstalled PyYAML-6.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-hb4h5sfd
  Running command git clone -q https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-hb4h5sfd
    

In [2]:
#import some important utilities of Detectron2
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
torch:  1.12 ; cuda:  cu113
detectron2: 0.6


In [3]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
#import features for the bbox
import numpy as np
import cv2
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

from detectron2.structures import BoxMode

In [5]:
# Load original classification repo
!pip install timm
!git clone https://github.com/facebookresearch/mvit.git mvit
!curl -OL  https://github.com/facebookresearch/detectron2/blob/717ab9f0aeca216a2f800e43d705766251ba3a55/projects/MViTv2/configs/mask_rcnn_mvitv2_t_3x.pth # pretrained

# Load repo that related to detectron2

!python -m pip install -e mvit


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'mvit'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 130 (delta 61), reused 88 (delta 22), pack-reused 0
Receiving objects: 100% (130/130), 65.00 KiB | 6.50 MiB/s, done.
Resolving deltas: 100% (61/61), done.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  206k    0  206k    0     0  1041k      0 --:--:-- --:--:-- --:--:-- 1041k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mvit
     |████████████████████████████████| 130 kB 5.2 MB/s 
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=efe924627a6e894cb096e258edf453c01b2eaac80df1208b3d72183c78

In [6]:
!rm -r mvit_detectron2
!git clone https://github.com/HeqingZ/FCOS_SET.git mvit_detectron2

rm: cannot remove 'mvit_detectron2': No such file or directory
Cloning into 'mvit_detectron2'...
remote: Enumerating objects: 24, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 24 (delta 11), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (24/24), done.


In [7]:
!pip install fcos

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 152 kB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for fcos: filename=fcos-0.9.8-cp37-cp37m-linux_x86_64.whl size=2775924 sha256=5d24c31f9df8d2a0ad5f2616a63e8b68ac53dab9d9710d00f965efdea0d5b043
  Stored in directory: /root/.cache/pip/wheels/a6/14/ec/911ce7084df67b170fa4ea3fa6312381f3f60eae5af41d05ad
  Created wheel for pycocotools: filename=pycocotools-2.0.1-cp37-cp37m-linux_x86_64.whl size=265253 sha256=7b4013343c3ff4164096af8a5bf082a8ae4930fea90663cc04a68eb66aae0d0e
  Stored in directory: /root/.cache/pip/wheels/12/36/32/2bae0d429403617ca371aa0a08b16f8746a09a5f56a3ae5ce1
Successfully built fcos pycocotools
  Attempting uninstall: pycocotools
    Found existing installation: pycocotools 2.0.4
    Uninstalling pycocotools-2.0.4:
      Successf

In [8]:
os.getcwd()

'/content'

In [9]:
%cd /content/

/content


In [4]:
####
# Train Here
####
!python mvit_detectron2/lazyconfig_train_net.py --config-file mvit_detectron2/fcos_configs_3x.py --resume

{'backbone': {'bottom_up': {'embed_dim': 96, 'depth': 10, 'num_heads': 1, 'last_block_indexes': [0, 2, 7, 9], 'residual_pooling': True, 'drop_path_rate': 0.2, 'norm_layer': functools.partial(<class 'torch.nn.modules.normalization.LayerNorm'>, eps=1e-06), 'out_features': ['scale2', 'scale3', 'scale4', 'scale5'], '_target_': <class 'detectron2.modeling.backbone.mvit.MViT'>}, 'in_features': '${.bottom_up.out_features}', 'out_channels': 256, 'top_block': {'in_channels': 256, 'out_channels': '${..out_channels}', '_target_': <class 'detectron2.modeling.backbone.fpn.LastLevelP6P7'>, 'in_feature': 'p5'}, '_target_': <class 'detectron2.modeling.backbone.fpn.FPN'>}, 'head': {'input_shape': [{'channels': 256, 'height': None, 'width': None, 'stride': None}, {'channels': 256, 'height': None, 'width': None, 'stride': None}, {'channels': 256, 'height': None, 'width': None, 'stride': None}, {'channels': 256, 'height': None, 'width': None, 'stride': None}, {'channels': 256, 'height': None, 'width': Non